# 使用 statsmodels 实现简单线性回归

In [15]:
raw = pd.read_csv('./creditcard_exp.csv')
raw.head()

,id,Acc,avg_exp,avg_exp_ln,gender,Age,Income,Ownrent,Selfempl,dist_home_val,dist_avg_income,age2,high_avg,edu_class
0,19,1,1217.03,7.104168743,1,40,16.03515,1,1,99.93,15.932789,1600,0.102361,3
1,5,1,1251.5,7.132098111,1,32,15.84750,1,0,49.88,15.796316,1024,0.051184,2
2,95,0,,,1,36,8.40000,0,0,88.61,7.490000,1296,0.910000,1
3,86,1,856.57,6.752936042,1,41,11.47285,1,0,16.10,11.275632,1681,0.197218,3
4,50,1,1321.83,7.186772419,1,28,13.40915,1,0,100.39,13.346474,784,0.062676,2


In [16]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
id                 100 non-null int64
Acc                100 non-null int64
avg_exp            100 non-null object
avg_exp_ln         100 non-null object
gender             100 non-null int64
Age                100 non-null int64
Income             100 non-null float64
Ownrent            100 non-null int64
Selfempl           100 non-null int64
dist_home_val      100 non-null float64
dist_avg_income    100 non-null float64
age2               100 non-null int64
high_avg           100 non-null float64
edu_class          100 non-null int64
dtypes: float64(4), int64(8), object(2)
memory usage: 11.0+ KB


In [17]:
raw['avg_exp'] = raw['avg_exp'].replace(' ',np.nan)

In [18]:
exp = raw[raw['avg_exp'].notnull()].iloc[:, 2:].drop('age2', axis=1)
exp.head()

,avg_exp,avg_exp_ln,gender,Age,Income,Ownrent,Selfempl,dist_home_val,dist_avg_income,high_avg,edu_class
0,1217.03,7.104168743,1,40,16.03515,1,1,99.93,15.932789,0.102361,3
1,1251.5,7.132098111,1,32,15.84750,1,0,49.88,15.796316,0.051184,2
3,856.57,6.752936042,1,41,11.47285,1,0,16.10,11.275632,0.197218,3
4,1321.83,7.186772419,1,28,13.40915,1,0,100.39,13.346474,0.062676,2
5,816.03,6.704451119,1,41,10.03015,0,1,119.76,10.332263,-0.302113,3


In [19]:
exp_new = raw[raw['avg_exp'].isnull()].iloc[:, 2:].drop('age2', axis=1)
exp_new.head()

,avg_exp,avg_exp_ln,gender,Age,Income,Ownrent,Selfempl,dist_home_val,dist_avg_income,high_avg,edu_class
2,NaN,,1,36,8.40,0,0,88.61,7.49,0.91,1
11,NaN,,1,22,5.10,0,0,109.07,4.78,0.32,1
13,NaN,,0,27,4.90,0,1,53.93,5.43,-0.53,1
19,NaN,,0,41,4.39,0,1,97.06,4.65,-0.26,1
20,NaN,,1,54,4.18,0,0,10.01,4.59,-0.41,0


In [20]:
exp_new.shape

(30, 11)

### 数据类型转换

In [21]:
exp['avg_exp'] = exp['avg_exp'].astype('float64')
exp['avg_exp_ln'] = exp['avg_exp_ln'].astype('float64')

In [22]:
exp.describe(include='all')

,avg_exp,avg_exp_ln,gender,Age,Income,Ownrent,Selfempl,dist_home_val,dist_avg_income,high_avg,edu_class
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,983.655429,6.787787,0.285714,31.157143,7.424706,0.385714,0.028571,74.540857,8.005472,-0.580766,1.928571
std,446.294237,0.476035,0.455016,7.206349,3.077986,0.490278,0.167802,36.949228,3.070744,0.432808,0.873464
min,163.180000,5.094854,0.000000,20.000000,3.493900,0.000000,0.000000,13.130000,3.828842,-1.526850,0.000000
25%,697.155000,6.547003,0.000000,26.000000,5.175662,0.000000,0.000000,49.302500,5.915553,-0.887981,1.000000
50%,884.150000,6.784627,0.000000,30.000000,6.443525,0.000000,0.000000,65.660000,7.084184,-0.612068,2.000000
75%,1229.585000,7.114415,1.000000,36.000000,8.494237,1.000000,0.000000,105.067500,9.123105,-0.302082,3.000000
max,2430.030000,7.795659,1.000000,55.000000,16.900150,1.000000,1.000000,157.900000,18.427000,0.259337,3.000000


In [23]:
exp[['Income', 'avg_exp', 'Age', 'dist_home_val']].corr()

,Income,avg_exp,Age,dist_home_val
Income,1.000000,0.674011,0.369129,0.249153
avg_exp,0.674011,1.000000,0.258478,0.319499
Age,0.369129,0.258478,1.000000,0.109323
dist_home_val,0.249153,0.319499,0.109323,1.000000


In [24]:
from statsmodels.formula.api import ols

lm_s = ols('avg_exp ~ Income', data=exp).fit()

In [25]:
print(lm_s.params)

Intercept    258.049498
Income        97.728578
dtype: float64


什么意思？

$$
Y = 258.049498 + 97.728578 \; X_1 + \eplson
$$

In [26]:
lm_s.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                avg_exp   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     56.61
Date:                Thu, 28 Feb 2019   Prob (F-statistic):           1.60e-10
Time:                        18:54:04   Log-Likelihood:                -504.69
No. Observations:                  70   AIC:                             1013.
Df Residuals:                      68   BIC:                             1018.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    258.0495    104.290      2.474      0.016      49.942     466.157
Income        97.7286     12.989      7.524      0.000      71.809     123.648
==============================================================================
Omnibus:                        3.714   Durbin-Watson:                   1.424
Prob(Omnibus):                  0.156   Jarque-Bera (JB):                3.507
Skew:                           0.485   Prob(JB):                        0.173
Kurtosis:                       2.490   Cond. No.                         21.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
pd.DataFrame([lm_s.predict(exp), lm_s.resid], index=['predict', 'resid']).T.head()

,predict,resid
0,1825.141904,-608.111904
1,1806.803136,-555.303136
3,1379.274813,-522.704813
4,1568.506658,-246.676658
5,1238.281793,-422.251793


In [29]:
from statsmodels.formula.api import ols

lm_m = ols('avg_exp ~ Age + Income + dist_home_val + dist_avg_income',data = exp).fit()

In [30]:
lm_m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                avg_exp   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     19.20
Date:                Thu, 28 Feb 2019   Prob (F-statistic):           1.82e-10
Time:                        18:54:15   Log-Likelihood:                -498.59
No. Observations:                  70   AIC:                             1007.
Df Residuals:                      65   BIC:                             1018.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -32.0078    186.874     -0.171      0.865    -405.221     341.206
Age                 1.3723      5.605      0.245      0.807      -9.822      12.566
Income           -166.7204     87.607     -1.903      0.061    -341.684       8.243
dist_home_val       1.5329      1.057      1.450      0.152      -0.578       3.644
dist_avg_income   261.8827     87.807      2.982      0.004      86.521     437.245
==============================================================================
Omnibus:                        5.234   Durbin-Watson:                   1.582
Prob(Omnibus):                  0.073   Jarque-Bera (JB):                5.174
Skew:                           0.625   Prob(JB):                       0.0752
Kurtosis:                       2.540   Cond. No.                         459.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

可以看出，变量 `Age` 和 `dist_home_val` 的 p-值都比较大，表示不显著，而 `Income` 和 `dist_avg_income` 显著。

$R_2$ 的值为 $0.542$，调整 $R_2$ 的值为 $0.513$。